# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [3]:
#Verificando variávels nulls
df.isnull().sum()

Unnamed: 0                  0
data_ref                    0
id_cliente                  0
sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.isnull().sum()

Unnamed: 0               0
data_ref                 0
id_cliente               0
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

## 1 - Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [6]:
# Lista de colunas do tipo 'object' que são as variáveis categóricas , excluindo 'data_ref'
variaveis_qualitativas = df.drop(columns=['data_ref']).select_dtypes(include='object').columns
print(variaveis_qualitativas)

Index(['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], dtype='object')


In [7]:
for col in df.columns:
    if col in variaveis_qualitativas:
        print(col, '->', df[col].unique())

sexo -> ['F' 'M']
tipo_renda -> ['Empresário' 'Assalariado' 'Servidor público' 'Bolsista' 'Pensionista']
educacao -> ['Secundário' 'Superior completo' 'Superior incompleto' 'Primário'
 'Pós graduação']
estado_civil -> ['Solteiro' 'Casado' 'Viúvo' 'União' 'Separado']
tipo_residencia -> ['Casa' 'Governamental' 'Com os pais' 'Aluguel' 'Estúdio' 'Comunitário']


In [8]:
# Verficando os niveis das categorias mais frequentes
print(df['tipo_renda'].value_counts())
print(df['educacao'].value_counts())
print(df['estado_civil'].value_counts())
print(df['tipo_residencia'].value_counts())
print(df['sexo'].value_counts())


Assalariado         7633
Empresário          3508
Servidor público    1268
Bolsista               9
Pensionista            9
Name: tipo_renda, dtype: int64
Secundário             7045
Superior completo      4695
Superior incompleto     558
Primário                103
Pós graduação            26
Name: educacao, dtype: int64
Casado      8897
Solteiro    1543
União        924
Separado     739
Viúvo        324
Name: estado_civil, dtype: int64
Casa             11071
Com os pais        674
Governamental      360
Aluguel            183
Estúdio             75
Comunitário         64
Name: tipo_residencia, dtype: int64
F    7901
M    4526
Name: sexo, dtype: int64


In [9]:
# Usando os niveis mais frequente como casela de referência para cada variável categórica
y, X = patsy.dmatrices('''np.log(renda) 
                    ~ C(sexo) 
                    + posse_de_veiculo 
                    + posse_de_imovel
                    + qtd_filhos 
                    + C(tipo_renda, Treatment('Assalariado')) 
                    + C(educacao, Treatment('Secundário')) 
                    + C(estado_civil, Treatment('Casado')) 
                    + C(tipo_residencia, Treatment('Casa')) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia
                    + 1''', df)



reg = sm.OLS(y,X).fit()

print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sat, 13 Jan 2024   Prob (F-statistic):               0.00
Time:                        08:26:28   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

O modelo parece apresentar uma capacidade pouca razoável de explicar a variação na renda com base nas variáveis incluídas, pois o R-squared e Adj. R-squared ambos em torno de 36%, indicam que aproximadamente 36% da variabilidade na renda é explicada pelas variáveis independentes. 

Além disso, o sexo masculino está associado a um log de renda 0.7874 maior do que o sexo feminino. Além disso, algumas categorias de variáveis categóricas, como tipo de renda, estado civil e tipo de residência, mostram efeitos significativos na previsão do log de renda.

Para os parâmetros que apresentaram coeficientes negativos, como é o caso de C(tipo_renda, Treatment('Assalariado'))[T.Pensionista], o valor indica uma associação negativa com o log de renda. Isso sugere que, mantendo todas as outras variáveis constantes, a categoria 'Pensionista' está associada a um log de renda menor em comparação com a categoria de referência ('Assalariado'). 

E seguida o p-value abaixo estão as variáveis que são estatísticamente insginificantes

In [10]:
# Extraindo variáveis não significativas com p-value acima de 0.05
insignificant_vars = [i for i, pvalue in enumerate(reg.pvalues) if pvalue > 0.05]

# Visualizando as variáveis não significativas e seus p-values
print("Variáveis não significativas:\n")
for i in insignificant_vars:
    print(f"{X.design_info.column_names[i]}: {reg.pvalues[i]:.2f}")



Variáveis não significativas:

C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]: 0.36
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]: 0.20
C(educacao, Treatment('Secundário'))[T.Primário]: 0.84
C(educacao, Treatment('Secundário'))[T.Pós graduação]: 0.39
C(educacao, Treatment('Secundário'))[T.Superior incompleto]: 0.37
C(estado_civil, Treatment('Casado'))[T.União]: 0.17
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]: 0.41
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]: 0.53
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]: 0.41
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]: 0.19
C(tipo_residencia, Treatment('Casa'))[T.Governamental]: 0.74


In [11]:
for i in insignificant_vars:
    print(X.design_info.column_names[i])

C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]
C(educacao, Treatment('Secundário'))[T.Primário]
C(educacao, Treatment('Secundário'))[T.Pós graduação]
C(educacao, Treatment('Secundário'))[T.Superior incompleto]
C(estado_civil, Treatment('Casado'))[T.União]
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]
C(tipo_residencia, Treatment('Casa'))[T.Governamental]


# 2 Remova a variável menos significante e analise:

In [12]:
# Lista de valores a serem excluídos
excluded_values = ['Bolsista', 'Pensionista', 'Primário', 'Pós graduação', 'Superior incompleto', 'União', 'Aluguel',
                     'Com os pais', 'Comunitário', 'Estudio', 'Governamental']

In [13]:
# Excluindo linhas que contenham os valores específicos em qualquer coluna e criando um novo DataFrame
df_new = df[~df.isin(excluded_values).any(axis=1)]

In [14]:
# Visualizando do novo DataFrame resultante
print("DataFrame sem os valores insignificantes:")
df_new.head()

DataFrame sem os valores insignificantes:


,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
5,5,2015-01-01,10581,F,False,True,0,Assalariado,Superior completo,Casado,Casa,39,4.345205,2.0,1445.87


In [15]:
df_new.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [16]:
# Lista de colunas do tipo 'object' que são as variáveis categóricas , excluindo 'data_ref'
var_new = df_new.drop(columns=['data_ref']).select_dtypes(include='object').columns
print(var_new)

Index(['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], dtype='object')


In [17]:
for col in df_new.columns:
    if col in var_new:
        print(col, '->', df_new[col].unique())

sexo -> ['F' 'M']
tipo_renda -> ['Empresário' 'Assalariado' 'Servidor público']
educacao -> ['Secundário' 'Superior completo']
estado_civil -> ['Solteiro' 'Casado' 'Viúvo' 'Separado']
tipo_residencia -> ['Casa' 'Estúdio']


In [18]:
# Verficando os niveis das categorias mais frequentes
print(df_new['tipo_renda'].value_counts())
print(df_new['educacao'].value_counts())
print(df_new['estado_civil'].value_counts())
print(df_new['tipo_residencia'].value_counts())
print(df_new['sexo'].value_counts())

Assalariado         6021
Empresário          2724
Servidor público    1040
Name: tipo_renda, dtype: int64
Secundário           5872
Superior completo    3913
Name: educacao, dtype: int64
Casado      7631
Solteiro    1218
Separado     644
Viúvo        292
Name: estado_civil, dtype: int64
Casa       9717
Estúdio      68
Name: tipo_residencia, dtype: int64
F    6282
M    3503
Name: sexo, dtype: int64


In [19]:
# Usando os niveis mais frequente como casela de referência para cada variável categórica
y_new, X_new = patsy.dmatrices('''np.log(renda)
                    ~ C(sexo)
                    + posse_de_veiculo
                    + posse_de_imovel
                    + qtd_filhos
                    + C(tipo_renda, Treatment('Assalariado'))
                    + C(educacao, Treatment('Secundário'))
                    + C(estado_civil, Treatment('Casado'))
                    + C(tipo_residencia, Treatment('Casa'))
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia
                    + 1''', df_new)



reg_new = sm.OLS(y_new, X_new).fit()

In [20]:
print(reg_new.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     400.2
Date:                Sat, 13 Jan 2024   Prob (F-statistic):               0.00
Time:                        08:29:42   Log-Likelihood:                -10759.
No. Observations:                9785   AIC:                         2.155e+04
Df Residuals:                    9770   BIC:                         2.166e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

O novo modelo reg_new, baseado pelo vlaores do R-squared e Adj. R-squared não apresenta diferença, quando ao percentual da variável resposta que pode ser explicado. Porém o AIC apresentou mudança:
 * AIC: 2.719e+04 no primeiro modelo, 
 * AIC: 2.155e+04 no segundo modelo
 
 O AIC do segundo modelo apresenta menor valor, mostrando que esse novo modelo tem qualidade melhor que o primeiro, sendo mais simples que o primeiro.
 
 O p-value manteve seus valores de significância para as variáveis que continuaram no modelo

In [21]:
# Lista de valores a serem excluídos
excluded_val = ['Estudio']
# Excluindo linhas que contenham os valores específicos em qualquer coluna e criando um novo DataFrame
df_new2 = df_new[~df_new.isin(excluded_val).any(axis=1)]
# Visualizando do novo DataFrame resultante
print("DataFrame sem os valores insignificantes:")
df_new2.head()

DataFrame sem os valores insignificantes:


,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
5,5,2015-01-01,10581,F,False,True,0,Assalariado,Superior completo,Casado,Casa,39,4.345205,2.0,1445.87


In [22]:
# Lista de colunas do tipo 'object' que são as variáveis categóricas , excluindo 'data_ref'
var_new2 = df_new.drop(columns=['data_ref']).select_dtypes(include='object').columns
print(var_new2)

Index(['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], dtype='object')


In [23]:
# Usando os niveis mais frequente como casela de referência para cada variável categórica
y_new2, X_new2 = patsy.dmatrices('''np.log(renda)
                    ~ C(sexo)
                    + posse_de_veiculo
                    + posse_de_imovel
                    + qtd_filhos
                    + C(tipo_renda, Treatment('Assalariado'))
                    + C(educacao, Treatment('Secundário'))
                    + C(estado_civil, Treatment('Casado'))
                    + C(tipo_residencia, Treatment('Casa'))
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia
                    + 1''', df_new)



reg_new2 = sm.OLS(y_new2, X_new2).fit()
print(reg_new2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     400.2
Date:                Sat, 13 Jan 2024   Prob (F-statistic):               0.00
Time:                        08:44:58   Log-Likelihood:                -10759.
No. Observations:                9785   AIC:                         2.155e+04
Df Residuals:                    9770   BIC:                         2.166e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [25]:
df_new2['tipo_residencia'].unique()

array(['Casa', 'Estúdio'], dtype=object)

## Removendo a variável tipo_residenencia

In [27]:
# Usando os niveis mais frequente como casela de referência para cada variável categórica
y_new3, X_new3 = patsy.dmatrices('''np.log(renda)
                    ~ C(sexo)
                    + posse_de_veiculo
                    + posse_de_imovel
                    + qtd_filhos
                    + C(tipo_renda, Treatment('Assalariado'))
                    + C(educacao, Treatment('Secundário'))
                    + C(estado_civil, Treatment('Casado'))
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia
                    + 1''', df_new)



reg_new3 = sm.OLS(y_new3, X_new3).fit()
print(reg_new3.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     430.9
Date:                Sat, 13 Jan 2024   Prob (F-statistic):               0.00
Time:                        08:47:37   Log-Likelihood:                -10759.
No. Observations:                9785   AIC:                         2.155e+04
Df Residuals:                    9771   BIC:                         2.165e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

Ao remover todsas a variáveis cujo p-value é maior que 0.05 não foi obseservado melhoria no modelo.